<a href="https://colab.research.google.com/github/DanilaSidulin/Lab2/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crc64iso

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from crc64iso.crc64iso import crc64
from math import ceil, log2
from random import randint

In [ ]:
def checksum(text):
  return crc64(text)


def text_to_bits(text):
  return "".join(
      f"{ord(c):08b}"
      for c in text
    )


def bits_to_text(data):
  text = ""
  while data != "":
    byte = data[0:8]
    data = data[8:]
    text += chr(int(byte, 2))
  return text


def text_to_bits(text, encoding='utf-8', errors="replace"):
    bits = bin(int.from_bytes(text.encode(encoding, errors), 'big'))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))
  

def bits_to_text(bits, encoding='utf-8', errors="replace"):
    n = int(bits, 2)
    return n.to_bytes((n.bit_length() + 7) // 8, 'big').decode(encoding, errors) or '\0'

In [ ]:
def calcRedundantBits(m):
	for i in range(m):
		if(2**i >= m + i + 1):
			return i

def posRedundantBits(data, r):
	j = 0
	k = 1
	m = len(data)
	res = ''
	for i in range(1, m + r+1):
		if(i == 2**j):
			res = res + '0'
			j += 1
		else:
			res = res + data[-1 * k]
			k += 1
	return res


def calcParityBits(arr, r): #Добавление контрольных битов
	n = len(arr)
	for i in range(r):
		val = 0
		for j in range(1, n + 1):
			if(j & (2**i) == (2**i)):
				val = val ^ int(arr[-1 * j])
		arr = arr[:n-(2**i)] + str(val) + arr[n-(2**i)+1:]
	return arr


def detectError(arr, nr): #Обнаружение ошибок
	n = len(arr)
	res = 0
	for i in range(nr):
		val = 0
		for j in range(1, n + 1):
			if(j & (2**i) == (2**i)):
				val = val ^ int(arr[-1 * j])
		res = res + val*(10**i)
	return int(str(res), 2)


def remove_redundant_bits(data): #Удаление лишних битов
  r_idx = 1
  i_to_remove = []

  while r_idx+1 < len(data):
    i_to_remove.append(r_idx-1)
    r_idx *= 2

  i_to_remove = i_to_remove[::-1]

  for i in i_to_remove:
    data = data[:i] + data[i+1:]

  return data

In [ ]:
def encode_word(data):
  length = len(data)
  red_count = calcRedundantBits(length)
  data_with_zeros = posRedundantBits(data[::-1], red_count)
  data_with_pars = calcParityBits(data_with_zeros[::-1], red_count)
  return data_with_pars[::-1]


def decode_word(word_data):
  return remove_redundant_bits(word_data)


def decode_word_error(word_data):
  decoded_word_data = remove_redundant_bits(word_data)
  r = calcRedundantBits(len(decoded_word_data))
  idx_error = detectError(word_data[::-1], r) - 1
  if idx_error == -1 or idx_error >= len(word_data):
    return decoded_word_data, 0
  word_data = word_data[:idx_error] + ("0" if word_data[idx_error] == "1" else "1") + word_data[idx_error+1:]
  return remove_redundant_bits(word_data), 1

In [ ]:
def encode(text, word_length):
  data = text_to_bits(text)
  data_length = len(data)
  num_blocks = ceil(data_length / word_length)
  data_blocks = [
      encode_word(data[i*word_length:(i+1)*word_length])
      for i in range(num_blocks)
  ]

  return data_blocks, checksum(text)


def decode(words_data):
  decoded_data = "".join(
      decode_word(word_data)
      for word_data in words_data
  )
  decoded_text = bits_to_text(decoded_data)
  return decoded_text, checksum(decoded_text)


def decode_error(words_data):
  decoded_data = ""
  errors = 0
  for word_data in words_data:
    decoded_word, error = decode_word_error(word_data)
    decoded_data += decoded_word
    errors += error
  decoded_text = bits_to_text(decoded_data)
  return decoded_text, errors, checksum(decoded_text)

In [ ]:
def place_error(words_data, word_num, bit_num):
  word_data = words_data[word_num]
  word_data = word_data[:bit_num] + ("0" if word_data[bit_num] == "1" else "1") + word_data[bit_num+1:]
  words_data[word_num] = word_data
  return words_data


def place_random_errors(words_data, max_errors_per_word):
  for word_num in range(len(words_data)):
    amount_of_errors  = randint(0, 1)
    if add_error:
      amount_of_errors = randint()
      place_error(words_data, word_num, randint(0, len(words_data[word_num])))

In [ ]:
text = "Что такое химия? Обратимся к википедии! Chemistry - the basis of such a chemical transformation is the rearrangement of electrons in the chemical bonds between atoms. It can be symbolically depicted through a chemical equation, which usually involves atoms as subjects. The number of atoms on the left and the right in the equation for a chemical transformation is equal. (When the number of atoms on either side is unequal, the transformation is referred to as a nuclear reaction or radioactive decay.) The type of chemical reactions a substance may undergo and the energy changes that may accompany it are constrained by certain basic rules, known as chemical laws. Energy and entropy considerations are invariably important in almost all chemical studies. Chemical substances are classified in terms of their structure, phase, as well as their chemical compositions. They can be analyzed using the tools of chemical analysis, e.g. spectroscopy and chromatography. Scientists engaged in chemical research are known as chemists. Most chemists specialize in one or more sub-disciplines. A chemical element is a pure substance which is composed of a single type of atom, characterized by its particular number of protons in the nuclei of its atoms, known as the atomic number and represented by the symbol Z. The mass number is the sum of the number of protons and neutrons in a nucleus. Although all the nuclei of all atoms belonging to one element will have the same atomic number, they may not necessarily have the same mass number; atoms of an element which have different mass numbers are known as isotopes. For example, all atoms with 6 protons in their nuclei are atoms of the chemical element carbon, but atoms of carbon may have mass numbers of 12 or 13. The standard presentation of the chemical elements is in the periodic table, which orders elements by atomic number. The periodic table is arranged in groups, or columns, and periods, or rows. The periodic table is useful in identifying periodic trends. A compound is a pure chemical substance composed of more than one element. The properties of a compound bear little similarity to those of its elements."
word_length = 101

In [ ]:
def main(text, word_length, max_broken_bits_per_word):
  encoded_blocks, check_init = encode(text, word_length)

  encoded_blocks, broken_words, errors_info = place_random_errors(encoded_blocks, max_broken_bits_per_word)
  print(f"Amount of broken words: {broken_words}")
  print(f"Amount of broken bits: {len(errors_info)}")

  decoded_text, check_dec = decode(encoded_blocks)
  print(f"Decoded text: {decoded_text}")
  print(f"Is the same? {decoded_text == text}")

  decoded_text_err, found_error_words, check_err = decode_error(encoded_blocks)
  print(f"Decoded text (fixing errors): {decoded_text_err}")
  print(f"Is the same? {decoded_text_err == text}")

  print(f"Found {found_error_words} broken words of {len(encoded_blocks)} total.")
  print(f"Found all errors? {len(errors_info) == found_error_words}")
  print(f"Decoding checksum: {check_init == check_dec}\nDecoding with errors fixed checksum: {check_init == check_err}")


Вызов без ошибок

In [ ]:
main(text, word_length, max_broken_bits_per_word=0) # Вызов без ошибок

Amount of broken words: 0
Amount of broken bits: 0
Decoded text: Что такое химия? Обратимся к википедии! Chemistry - the basis of such a chemical transformation is the rearrangement of electrons in the chemical bonds between atoms. It can be symbolically depicted through a chemical equation, which usually involves atoms as subjects. The number of atoms on the left and the right in the equation for a chemical transformation is equal. (When the number of atoms on either side is unequal, the transformation is referred to as a nuclear reaction or radioactive decay.) The type of chemical reactions a substance may undergo and the energy changes that may accompany it are constrained by certain basic rules, known as chemical laws. Energy and entropy considerations are invariably important in almost all chemical studies. Chemical substances are classified in terms of their structure, phase, as well as their chemical compositions. They can be analyzed using the tools of chemical analysis, e.g. s

Вызов с 1 ошибкой

In [ ]:
main(text, word_length, max_broken_bits_per_word=1) #Вызов с 1 ошибкой в слове

Amount of broken words: 89
Amount of broken bits: 89
Decoded text: Что قак��е химия? Обрати��ся к википедиа! Chemistry - the basis of such a`chemical transformation is the rearrangement`of electrons in the(chemical bonds between atoms. It can je symbolically depicted through c chemical equation, which usually involveS atoms as subjects. The number of atoms oN the levt and the right in the equation for a chelical transformatikn is equal. (When the number of atoos on either side is unequal, the transformation is referref to as a nucleas reaction or radioactive decay.) The type of chemikal rea#tions a sub3tance may undergo an` the energy changes thau may iccompany it are constrahned by certain basic rtles, known as chemical laws. Energy an� entropy conriderationq are invariably important in almost all chamical studies. Chemic!l substances are classified in terms of their struct�re, phase. as well as thair chemical compositions. Thei can be analy~ed�using the tools of chemical analysis, e.

In [139]:
main(text, word_length, max_broken_bits_per_word=2) #Вызов более чем с одной ошибкой в слове

Amount of broken words: 110
Amount of broken bits: 160
Decoded text: ЧQ�о тако�50химиُ? Обратимщя к википедйи! Chemistr{ - the basis of!�uch a cheoical transformatio~ is the reirrangem�nt of ele�tbons in the chdmical bonds betseen atoms. It can be symbodically depicted thro}gH a chemical equation, which usually involves atoms!as su`jects. The�nuiber of atoms on the heft and tje righd �n the euuation for a ch%micql transformation is epual. (When�the numcer od atoms on either side ys unequal,�txe transformatiOn is�r%ferped to as a nuclear Reaction or ralioactive!decay.) The�type of chemical rdactions c su"stance may undergo and the energy changes that may accompani it are constrained by cevtain basic rules, known as cxemica| laws. Energy ajd entropy considErations ar% inrariably important in aloost`aml chemical s�udies. Chemican substances are0clawsified in terms`of thEir stbucture, phasg, As well as their chemica� compowithons. They can re analyzed using the tools of ghgmigal analysis, 